# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [3]:
valuation_date = Date(2, 8, 2007)
settlement_date = valuation_date.add_weekdays(1)

## Build Ibor Curve

In [5]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(valuation_date, "1D", 0.0500, dcType); depos.append(depo)
depo = IborDeposit(settlement_date, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settlement_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [6]:
step_in_date = valuation_date.add_weekdays(1)

In [7]:
maturity3Y = valuation_date.next_cds_date(36)
maturity5Y = valuation_date.next_cds_date(60)
maturity7Y = valuation_date.next_cds_date(84)
maturity10Y = valuation_date.next_cds_date(120)

### Heterogeneous Curves

In [9]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [10]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40 

In [11]:
cdsIndex = CDSIndexPortfolio()

In [12]:
spd3Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity10Y, heteroIssuerCurves)

In [13]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.67879118619867
Homogeneous curve 5Y: 35.53904916872567
Homogeneous curve 7Y: 49.01163746623519
Homogeneous curve 10Y: 61.413389420329636


In [16]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Tranches

In [17]:
trancheMaturity = maturity5Y
tranche1 = CDSTranche(valuation_date, trancheMaturity, 0.00, 0.03)
tranche2 = CDSTranche(valuation_date, trancheMaturity, 0.03, 0.06)
tranche3 = CDSTranche(valuation_date, trancheMaturity, 0.06, 0.09)
tranche4 = CDSTranche(valuation_date, trancheMaturity, 0.09, 0.12)
tranche5 = CDSTranche(valuation_date, trancheMaturity, 0.12, 0.22)
tranche6 = CDSTranche(valuation_date, trancheMaturity, 0.22, 0.60)
tranche7 = CDSTranche(valuation_date, trancheMaturity, 0.00, 0.60)
tranche8 = CDSTranche(valuation_date, trancheMaturity, 0.00, 1.00)

In [18]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [19]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [20]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))
for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(valuation_date,homoIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, num_points, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.142344
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.142344
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   904.131917
                    FinLossDistributionBuilder.LHP    50   0.00000   3.00000   914.428151
              FinLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.744106
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.744106
               FinLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   215.897269
                    FinLossDistributionBuilder.LHP    50   3.00000   6.00000   226.978741
              FinLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.226143
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   9.00000   102.226143
         

## Heterogeneous Portfolio Results

In [21]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))

for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(valuation_date,heteroIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, num_points, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.643014
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.577467
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   981.024090
                    FinLossDistributionBuilder.LHP    50    0.00000   3.00000   914.815317
              FinLossDistributionBuilder.RECURSION    50    3.00000   6.00000   231.078872
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   231.166682
               FinLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   202.418752
                    FinLossDistributionBuilder.LHP    50    3.00000   6.00000   226.986598
              FinLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.300805
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    6.00000   9.00000    87.355840

Copyright (c) 2020 Dominic O'Kane